In [ ]:
!pip install sentence-transformers faiss-cpu transformers gradio feedparser streamlit
!pip install langchain langchain-community langgraph



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━

In [ ]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr
import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langgraph.graph import StateGraph
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [ ]:
class MCPServer:
    def __init__(self):
        self.context_history = []

    def save_context(self, query, summary, recommendations):
        self.context_history.append({
            "query": query,
            "summary": summary,
            "recommendations": recommendations
        })

    def get_latest(self):
        return self.context_history[-1] if self.context_history else None

    def export_topics_csv(self, filename="topics.csv"):
      queries = [ctx["query"] for ctx in self.context_history]
      df = pd.DataFrame({"topics": queries})
      df.to_csv(filename, index=False)
      return filename


# Global server object
mcp_server = MCPServer()

In [ ]:
#import feedparser
#from urllib.parse import quote

#def search_arxiv(query, max_results=20):
#    base_url = 'http://export.arxiv.org/api/query?'
#    search_query = f'search_query=all:{quote(query)}&start=0&max_results={max_results}'
#    feed = feedparser.parse(base_url + search_query)
#    return feed.entries

#arxiv_tool = Tool(
#    name="arxiv_search",
#    func=lambda q: search_arxiv(q),
#    description="Search ArXiv for papers by query string"
#)



In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="t5-base")
print("✅ Summarizer model loaded!")

def summarize_text(text):
    return summarizer(text, max_length=200, min_length=50, do_sample=False)[0]["summary_text"]




# Define summarizer function
def summarize_input_text(text, chunk_size=512):
    if not text or len(text.strip()) < 20:
        return "No summary available."

    # Clean and split
    words = text.replace("\n", " ").replace("-", " ").strip().split()

    # Chunking into multiple pieces
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

    summaries = []
    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=200,
            min_length=30,
            do_sample=False
        )
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries)

summarizer_tool = Tool(
    name="summarizer",
    func=summarize_input_text,
    description="Summarizes long academic abstracts"
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


✅ Summarizer model loaded!


In [ ]:
import pandas as pd

# Load dataset if previously saved
df_loaded = pd.read_csv('Final_Realistic_Legal_Cases.csv')
df_loaded['clean_text'] = df_loaded['clean_text'].fillna("")
print(f"✅ Loaded CSV with {df_loaded.shape[0]} papers.")
display(df_loaded.head())

✅ Loaded CSV with 1200 papers.


,title,summary,link,published,court,domain,subdomain,clean_text,token_count,case_year,case_id,jurisdiction,clean_title
0,"Davis v. IRS, 769 F.Supp.2d (2004)",This opinion addresses legal questions related...,https://www.courtlistener.com/opinion/5423652/,2019-07-05,"U.S. Court of Appeals, 9th Circuit",Corporate & Commercial Law,E-Commerce Law,The case deals with e-commerce law ruling. thi...,36,2019,2ab8b6e71b,US-State,"Davis v. IRS, 769 F.Supp.2d (2004)"
1,"Taylor v. EPA, 909 Fed. Appx. (2017)",This opinion addresses legal questions related...,https://www.courtlistener.com/opinion/5535264/,2024-04-15,"U.S. District Court, Southern District of New ...",Intellectual Property & Technology Law,Trademark Law,The opinion explores legal issues surrounding ...,33,2024,f6a3e6586a,US-State,"Taylor v. EPA, 909 Fed. Appx. (2017)"
2,"Roe v. Wade, 639 F.Supp.2d (2020)",This opinion addresses legal questions related...,https://www.courtlistener.com/opinion/7732493/,2021-07-12,Supreme Court of the United States,Intellectual Property & Technology Law,Digital Rights Management,This judgment concerns digital rights manageme...,40,2021,5ad5eeff4f,US-State,"Roe v. Wade, 639 F.Supp.2d (2020)"
3,"Martin v. Google Inc., 396 F.Supp.2d (2006)",This opinion addresses legal questions related...,https://www.courtlistener.com/opinion/9890303/,2023-05-08,Supreme Court of the United States,Environmental & Energy Law,Air Pollution Regulation,The case deals with air pollution regulation r...,35,2023,ad0e59b9ec,US-State,"Martin v. Google Inc., 396 F.Supp.2d (2006)"
4,"Davis v. IRS, 674 F.3d (2011)",This opinion addresses legal questions related...,https://www.courtlistener.com/opinion/6632109/,2018-06-02,"U.S. Court of Appeals, 9th Circuit",Constitutional & Administrative Law,Judicial Review,This ruling pertains to judicial review ruling...,32,2018,8cdfa9f5bf,US-State,"Davis v. IRS, 674 F.3d (2011)"


In [ ]:
from tqdm import tqdm

# Keep domain structure as is
domain_structure = {
    "Constitutional & Administrative Law": [
        "Administrative Tribunals", "Constitutional Amendments", "Emergency Powers", "Federalism",
        "Fundamental Rights", "Judicial Review", "Legislative Powers", "Public Interest Litigation",
        "Separation of Powers", "Writ Jurisdiction"
    ],
    "Corporate & Commercial Law": [
        "Banking Law", "Commercial Arbitration", "Company Law", "Competition Law", "Consumer Protection",
        "Contract Law", "E-Commerce Law", "Insolvency & Bankruptcy", "Mergers & Acquisitions", "Securities Regulation"
    ],
    "Criminal Law & Procedure": [
        "Bail & Sentencing", "Criminal Procedure", "Cyber Crime", "Double Jeopardy", "Evidence in Criminal Cases",
        "Juvenile Justice", "Search and Seizure", "Substantive Criminal Law", "Victim Rights", "White-Collar Crime"
    ],
    "Environmental & Energy Law": [
        "Air Pollution Regulation", "Climate Change Law", "Environmental Impact Assessment", "Environmental Litigation",
        "Forest Law", "Mining & Natural Resources Law", "Renewable Energy Law", "Sustainable Development", "Water Law", "Wildlife Protection"
    ],
    "Intellectual Property & Technology Law": [
        "AI & Law", "Copyright Law", "Data Privacy Law", "Digital Rights Management", "IP Licensing", "Internet Governance",
        "Patent Law", "Software Licensing", "Trade Secrets", "Trademark Law"
    ],
    "International & Human Rights Law": [
        "Climate Agreements", "Diplomatic Immunity", "Gender Rights under International Law", "Global Health Law",
        "Humanitarian Law", "International Criminal Court", "International Treaties", "Refugee Law", "UN Conventions", "War Crimes"
    ]
}

In [ ]:


# New function: fetch papers and summarize using MCP tools
#def fetch_and_summarize_papers(domain_structure, max_results=20):
    #all_papers = []
    #for domain, subdomains in domain_structure.items():
        #for sub in subdomains:
#             print(f"🔍 Fetching papers for: {domain} → {sub}")

#             # Use arxiv_tool to fetch papers
#             papers = arxiv_tool.run(sub)

#             for paper in papers:
#                 abstract_text = paper.get("summary", "")
#                 if abstract_text:
#                     # Use summarizer_tool to summarize abstract
#                     summarized = summarizer_tool.run(abstract_text)
#                 else:
#                     summarized = "No abstract available"

#                 paper['summary'] = summarized
#                 paper['domain'] = domain
#                 paper['subdomain'] = sub
#             all_papers.extend(papers)

#     df_all = pd.DataFrame(all_papers)
#     return df_all



# df_all = fetch_and_summarize_papers(domain_structure)
# df_all.to_csv("arxiv_1200_papers.csv", index=False)
# print("✅ Papers fetched, summarized, and saved to CSV!")

In [ ]:
# # Check paper counts per subdomain (optional - uncomment when needed)
# if 'df_all' in globals():
#     subdomain_counts = df_all.groupby(['domain', 'subdomain']).size().reset_index(name='num_papers')
#     print(subdomain_counts.sort_values('num_papers'))
# else:
#     print("⚠️ Dataframe 'df_all' not loaded yet. Run fetch_and_summarize_papers() or load CSV first.")


In [ ]:
# Load embedding model (already used in previous block but kept here if re-run separately)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded!")

# Define MCP Tool for embedding if not already defined
def embed_text(text):
    """Embed a single text string into a vector."""
    return embed_model.encode([text]).tolist()

embedder_tool = Tool(
    name="embedder",
    func=embed_text,
    description="Embeds text into vector representations"
)

# Build FAISS index using abstracts
abstract_texts = df_loaded['clean_text'].fillna("").tolist()
abstract_embeddings = embed_model.encode(abstract_texts, show_progress_bar=True)
abstract_embeddings = np.array(abstract_embeddings).astype('float32')

dimension = abstract_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(abstract_embeddings)
print(f"✅ FAISS index created with {index.ntotal} papers!")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded!


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ FAISS index created with 1200 papers!


In [ ]:
from transformers import pipeline

# Load zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

print(" Zero-shot classifier loaded!")
candidate_labels = [
    # AI
    "Machine Learning", "Deep Learning", "Reinforcement Learning", "Transfer Learning",
    "Self-Supervised Learning", "Meta-Learning", "Federated Learning", "Explainable AI",
    "Generative AI", "AI Ethics",

    # Healthcare
    "Medical Imaging", "Clinical NLP", "Drug Discovery", "Disease Diagnosis Models",
    "Predictive Healthcare", "Public Health Analytics", "Electronic Health Records",
    "Wearable Health Devices", "Mental Health AI", "COVID-19 Research",

    # Finance
    "Stock Price Prediction", "Fraud Detection", "Algorithmic Trading", "Credit Risk Modeling",
    "Financial Sentiment Analysis", "AI in Insurance", "Economic Forecasting",
    "Portfolio Optimization", "Blockchain & Crypto Analytics", "Regulatory Technology",

    # Law
    "Legal Document Summarization", "AI in Legal Reasoning", "Contract Analysis & NLP",
    "Legal QA", "Case Law Retrieval", "Judgment Prediction", "Legal Chatbots",
    "Privacy Law Compliance", "Policy Modeling",

    # Education
    "Intelligent Tutoring Systems", "AI in Education", "Student Performance Prediction",
    "Academic Plagiarism Detection", "Exam Question Generation", "Adaptive Learning",
    "Education Data Mining", "Misinformation in EdTech", "Educational NLP",
    "Conversational Agents in Education",

    # Sustainability & Robotics
    "Climate Modeling", "Renewable Energy", "Smart Grids", "AI in Agriculture",
    "Precision Farming", "Autonomous Vehicles", "Robotics & Control",
    "Predictive Maintenance", "Industrial Automation", "Disaster Response"
]

flat_subdomains = [sub for subs in domain_structure.values() for sub in subs]
domain_map = {sub: domain for domain, subs in domain_structure.items() for sub in subs}

def classify_legal_case(text):
    result = classifier(text, flat_subdomains)
    subdomain = result['labels'][0]
    domain = domain_map.get(subdomain, "Uncategorized")
    return domain, subdomain

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


 Zero-shot classifier loaded!


In [ ]:
def find_related_papers(user_input, top_k=5, domain_filter="All", subdomain_filter="All"):
    # Search FAISS index on the entire dataset
    user_vec = embed_model.encode([user_input]).astype('float32')
    D, I = index.search(user_vec, 50)  # fetch more than top_k initially

    results = []
    for idx, dist in zip(I[0], D[0]):
        if idx >= len(df_loaded):
            continue

        paper = df_loaded.iloc[idx].to_dict()

        # Apply filters AFTER search
        if domain_filter != "All" and paper['domain'] != domain_filter:
            continue
        if subdomain_filter != "All" and paper['subdomain'] != subdomain_filter:
            continue

        paper['similarity_score'] = round(float(dist), 3)
        results.append(paper)

        if len(results) == top_k:
            break

    return results




# Wrap as MCP tool
recommender_tool = Tool(
    name="recommender",
    func=lambda q: find_related_papers(q, top_k=5),
    description="Recommends related papers with similarity scores and metadata"
)


In [ ]:
def generate_csv(recommendations):
    """
    Save recommended papers to CSV.
    Now recommendations is a list of dicts (from find_related_papers) not a DataFrame.
    """
    if isinstance(recommendations, list):
        df_out = pd.DataFrame(recommendations)
    else:
        df_out = recommendations

    # Check if similarity_score exists in recommendations
    columns_to_save = ['title', 'summary', 'link', 'domain', 'subdomain']
    if 'similarity_score' in df_out.columns:
        columns_to_save.append('similarity_score')

    df_out = df_out[columns_to_save]
    df_out.to_csv("recommendations.csv", index=False)
    return "recommendations.csv"

In [ ]:
def summarizer_and_recommender(user_input):
    """
    Summarizes the input using the summarizer_tool and fetches related papers
    using the updated recommender_tool (which now returns full metadata).
    """
    if not user_input.strip():
        return "⚠️ Please enter some input text."

    # Summarize user input
    summary = summarizer_tool.run(user_input)

    # Get recommendations (list of dicts with metadata)
    recommendations = recommender_tool.run(user_input)

    # Build output string
    output = f"###  Your Input Summary:\n{summary}\n\n"
    output += "### Recommended Papers:\n\n"

    for rec in recommendations:
        output += f"**Title:** {rec.get('title', 'N/A')}  \n"


        if 'published' in rec:
            output += f"**Published:** {rec.get('published', 'N/A')}  \n"

        if 'link' in rec:
            output += f"**Link:** [View Paper]({rec.get('link', '#')})  \n"

        if 'domain' in rec:
            output += f"**Domain:** {rec.get('domain', 'N/A')}  \n"

        if 'similarity_score' in rec:
            output += f"**Similarity Score:** {rec.get('similarity_score', 0):.2f}  \n"

        output += f"**Summary:** {rec.get('summary', 'No summary available')}  \n"
        output += "---  \n\n"

    return output


In [ ]:
from langchain.tools import Tool

mcp_tool = Tool(
    name="MCP Summarizer Recommender",
    func=lambda q: mcp_summarize_and_recommend(q["query"], q.get("domain", "All"), q.get("subdomain", "All")),
    description="Summarizes research input and recommends papers with citations. Input format: {'query': ..., 'domain': ..., 'subdomain': ...}"
)

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Optimized inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer,
                max_length=512, do_sample=False)

# HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=pipe)

# Initialize the MCP Agent

agent = initialize_agent(
    tools=[mcp_tool],  # instead of google_scholar_tool and summarizer_tool
    llm = llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

print("✅ Agent with FLAN-T5-large ready!")




tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Agent with FLAN-T5-large ready!


/tmp/ipython-input-3760318709.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipython-input-3760318709.py:22: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/h

In [ ]:
# Define your input text (replace with anything you want to test)
user_input_text = """
The petitioner, a non-profit environmental advocacy group, has approached this Hon'ble Court under Article 32 of the Constitution of India, seeking enforcement of fundamental rights guaranteed under Articles 14 and 21. The grievance raised pertains to the indiscriminate and unregulated expansion of industrial activities in the ecologically sensitive region of the Western Ghats, a recognized UNESCO World Heritage Site. The petitioner contends that such development is being carried out in flagrant violation of the Environment Protection Act, 1986, the Forest (Conservation) Act, 1980, and the guidelines issued by the Ministry of Environment, Forest and Climate Change (MoEFCC). It is alleged that numerous clearances for setting up polluting industries, including thermal power plants, mining units, and chemical processing facilities, have been granted by the State Environmental Impact Assessment Authority (SEIAA) without mandatory environmental impact assessments (EIAs) and without following the public consultation process as mandated by the EIA Notification of 2006. The petitioner argues that this not only violates the statutory framework but also undermines the rights of tribal communities and forest dwellers who depend on these forests for their livelihood. They cite reports from the Comptroller and Auditor General of India and independent committees like the Gadgil Committee and the Kasturirangan Committee to substantiate their claim that regulatory oversight has been largely absent and that ecological degradation has accelerated over the last five years. The State Government, in its affidavit, contends that the industrial development is essential for economic progress and employment generation. It states that all clearances were issued as per the law, and necessary compensatory afforestation measures were taken. The affidavit further adds that while there may have been lapses in documentation or timing, the overall intent was aligned with the principles of sustainable development. The Union of India has taken a neutral stance, submitting that while policy formulation lies with the Centre, the implementation and monitoring of environmental clearances fall under the purview of the State Government and its subordinate agencies. During the course of arguments, the learned counsel for the petitioner relied heavily on the precautionary principle, the polluter pays principle, and the doctrine of public trust—all recognized as part of the environmental jurisprudence under Article 21, as laid down in landmark decisions such as Vellore Citizens’ Welfare Forum v. Union of India and M.C. Mehta v. Union of India. It was submitted that development cannot come at the cost of irreversible ecological damage and that procedural safeguards like public hearings and scientific assessments cannot be bypassed merely on the ground of expediency. On the other hand, the counsel for the State emphasized that industrial growth is critical for the State’s fiscal health and that the environmental safeguards were adhered to “substantially” if not “meticulously.” They also argued that PILs such as this must not paralyze executive decision-making and developmental planning. Reliance was placed on Centre for Public Interest Litigation v. Union of India, where the Hon’ble Supreme Court cautioned against judicial overreach in matters of policy unless there is a clear violation of constitutional or statutory provisions. In the backdrop of these arguments, this Court is called upon to examine the balance between economic development and environmental protection, especially in the context of the rights of indigenous communities, the statutory mandate of environmental laws, and the constitutional principles of sustainable development. The documents produced before us clearly indicate that several projects commenced operations even before receiving final environmental clearance. The MoEFCC’s own inspection reports reflect that critical wildlife corridors have been disrupted, water bodies have been polluted, and community health indicators have worsened in adjoining villages. The Court cannot ignore that the right to a clean and healthy environment is an integral part of the right to life under Article 21. Simultaneously, Article 19(1)(g) ensures the right to practice any profession or carry on any occupation, trade or business. However, both rights are subject to reasonable restrictions. The notion that economic advancement must come at the cost of environmental degradation is a fallacy that has been categorically rejected by this Court in Narmada Bachao Andolan v. Union of India. In this regard, it is also relevant to point out that the State’s failure to hold public consultations before approving Category A and B projects constitutes a procedural impropriety of significant magnitude. The process of public participation is not a mere formality but an essential component of environmental governance. The right to be heard, particularly of marginalized communities, is implicit in the broader constitutional framework of participatory democracy. Additionally, the Court finds that despite the existence of a detailed regulatory framework, including environmental clearance conditions, post-clearance monitoring has been abysmal. There is no evidence of consistent site inspections, pollution audits, or compliance reviews. The State’s claim of compensatory afforestation is also undermined by satellite imagery and ground reports indicating negligible forest regeneration. In fact, the State has not even maintained a geo-tagged inventory of afforestation projects, violating its obligations under the Forest Survey of India guidelines. Given these findings, this Court cannot remain a mute spectator to the environmental devastation that is unfolding under the garb of industrial development. Judicial intervention becomes not just justified but necessary to enforce constitutional mandates. Accordingly, the Court directs the following: All industrial clearances issued in the last 3 years within the notified eco-sensitive zones in the Western Ghats are stayed until a review is conducted by an independent expert panel. The State Government is directed to conduct cumulative environmental impact assessments (CEIA) in the affected regions within six months. A special monitoring committee comprising representatives from the judiciary, civil society, and environmental scientists is to be constituted to oversee compliance. The MoEFCC is directed to revise and update the eco-sensitive zone maps and notify the same within 90 days. This Court further emphasizes that while economic development is essential, it must be grounded in constitutional morality, scientific reasoning, and intergenerational equity. In the present case, the balance clearly tilts in favor of environmental conservation and the protection of the rights of forest-dependent communities. The writ petition is therefore allowed with the above directions.
"""
print(" USER INPUT TEXT")
print(user_input_text)
print("\n==============================\n")

# 1. Summarize input using the summarizer tool
user_summary = summarizer_tool.run(user_input_text)
print(" SUMMARIZED INPUT")
print(user_summary)
print("\n==============================\n")


# 2. Get recommendations using the recommender tool (full metadata)
recommendations = recommender_tool.run(user_input_text)

print(" RECOMMENDED LEGAL CASES")
for rec in recommendations:
    print(f"- Title: {rec.get('title', 'N/A')}")
    print(f"  Domain: {rec.get('domain', 'N/A')}")
    print(f"  Subdomain: {rec.get('subdomain', 'N/A')}")
    print(f"  Published: {rec.get('published', 'N/A')}")
    print(f"  Link: {rec.get('link', '#')}")
    print(f"  Similarity Score: {rec.get('similarity_score', 0):.2f}")
    print(f"  Case Text: {rec.get('clean_text', 'No case content available')}")
    print("--------------------------------")

 USER INPUT TEXT

The petitioner, a non-profit environmental advocacy group, has approached this Hon'ble Court under Article 32 of the Constitution of India, seeking enforcement of fundamental rights guaranteed under Articles 14 and 21. The grievance raised pertains to the indiscriminate and unregulated expansion of industrial activities in the ecologically sensitive region of the Western Ghats, a recognized UNESCO World Heritage Site. The petitioner contends that such development is being carried out in flagrant violation of the Environment Protection Act, 1986, the Forest (Conservation) Act, 1980, and the guidelines issued by the Ministry of Environment, Forest and Climate Change (MoEFCC). It is alleged that numerous clearances for setting up polluting industries, including thermal power plants, mining units, and chemical processing facilities, have been granted by the State Environmental Impact Assessment Authority (SEIAA) without mandatory environmental impact assessments (EIAs) an

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 SUMMARIZED INPUT
petitioner seeks enforcement of fundamental rights guaranteed under Articles 14 and 21 . indiscriminate expansion of industrial activities in ecologically sensitive region of the western ghats . petitioner argues that clearances for setting up polluting industries have been granted without mandatory environmental impact assessments . the Court cannot ignore that the right to a clean and healthy environment is an integral part of right to life under Article 21 . the notion that economic advancement must come at the cost of environmental degradation is a fallacy that has been categorically rejected by this Court .


 RECOMMENDED LEGAL CASES
- Title: Smith v. Johnson, 341 F.Supp.2d (2016)
  Domain: Environmental & Energy Law
  Subdomain: Environmental Impact Assessment
  Published: 2023-04-16
  Link: https://www.courtlistener.com/opinion/6252766/
  Similarity Score: 0.76
  Case Text: The court evaluates arguments related to environmental impact assessment ruling. in a si

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
def format_citation(title, published, link):
    year = published.split("-")[0] if published else "n.d."
    return f"<i>{title}</i> ({year}). Retrieved from <a href='{link}' target='_blank'>{link}</a>"




In [ ]:
def mcp_summarize_and_recommend(query, domain="All", subdomain="All"):
    try:
        summary = summarize_text(query)

        # 🔎 Filter papers based on domain/subdomain
        filtered_df = df_loaded.copy()
        if domain != "All":
            filtered_df = filtered_df[filtered_df['domain'] == domain]
        if subdomain != "All":
            filtered_df = filtered_df[filtered_df['subdomain'] == subdomain]

        if filtered_df.empty:
            return "No papers found for the selected filters."

        # 🔍 Embed the user query
        user_embedding = embed_model.encode([query]).astype("float32")

        # 🔁 Build temporary index for filtered subset
        subset_embeddings = embed_model.encode(filtered_df['clean-text'].fillna("").tolist()).astype("float32")
        faiss_index = faiss.IndexFlatL2(subset_embeddings.shape[1])
        faiss_index.add(subset_embeddings)

        distances, indices = faiss_index.search(user_embedding, min(5, len(filtered_df)))
        output = f"🔍 **Your Summary:**\n{summary}\n\n---\n\n"

        for idx, score in zip(indices[0], distances[0]):
            paper = filtered_df.iloc[idx]
            citation = format_citation(paper['title'], paper['published'], paper['link'])
            output += f"**Title:** {paper['title']}\n"
            output += f"**Published:** {paper['published']}\n"
            output += f"**Link:** {paper['link']}\n"
            output += f"**Similarity Score:** {score:.2f}\n"
            output += f"**Summary:** {paper['clean_text']}\n"
            output += f"**Citation:** {citation}\n"
            output += "---\n\n"

        return output

    except Exception as e:
        return f"⚠️ MCP Agent Error: {e}"



In [ ]:
def update_subdomains(domain):
    """Update subdomain dropdown based on selected domain."""
    if domain in domain_structure:
        return gr.update(choices=domain_structure[domain], value=None)
    return gr.update(choices=[], value=None)

def run_agent_ui(user_input, domain, subdomain):
    """Run summarizer + recommender with filtering."""
    if not user_input.strip():
        return "⚠️ Please enter an abstract.", "<p style='color:red;'>No papers found.</p>"

    summary = summarizer_tool.run(user_input)

    # Auto-classify if user left filters empty
    auto_domain, auto_subdomain = classify_legal_case(summary)
    domain_filter = domain if domain and domain.strip() != "" else auto_domain
    subdomain_filter = subdomain if subdomain and subdomain.strip() != "" else auto_subdomain

    # 2. Recommendations (always use the global df_loaded)
    recommendations = find_related_papers(
        user_input,
        top_k=5,
        domain_filter=domain_filter,
        subdomain_filter=subdomain_filter
    )

    if not recommendations or len(recommendations) == 0:
        return summary, "<p>No recommended papers found for this input.</p>"


    # Build HTML output
    papers_html = "<h3>Recommended Papers</h3>"

    for rec in recommendations:
      apa = format_citation(
        rec.get('title', 'N/A'),
        rec.get('published', 'n.d.'),
        rec.get('link', '#')
      )

      papers_html += f"""
      <div style="margin-bottom: 15px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
        <b>Title:</b> {rec.get('title', 'N/A')}<br>
        <b>Domain:</b> {rec.get('domain', 'N/A')}<br>
        <b>Published:</b> {rec.get('published', 'N/A')}<br>
        <b>Link:</b> <a href="{rec.get('link', '#')}" target="_blank">View Paper</a><br>
        <b>Similarity Score:</b> {rec.get('similarity_score', 0):.2f}<br>
        <b>Summary:</b> {rec.get('clean_text', 'No summary available')}<br><br>

        <details>
            <summary><b>Show Citation</b></summary>
            {apa}
        </details>
      </div>
      """
    csv_path = generate_csv(recommendations)

    mcp_server.save_context(user_input, summary, recommendations)


    return summary, papers_html, csv_path




# Build the Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## ⚖️ Legal Case Summarizer + Recommender Agent")

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Input Legal Case Text", lines=8, placeholder="Paste abstract here...")
            domain = gr.Dropdown(choices=list(domain_structure.keys()), label="Domain (Optional)", value=None)
            subdomain = gr.Dropdown(choices=[], label="Subdomain (Optional)", value=None)

            # Dynamic update of subdomains
            domain.change(fn=update_subdomains, inputs=domain, outputs=subdomain)

            submit_btn = gr.Button("▶️ Start Agent")

        with gr.Column():
            summary_output = gr.Textbox(label="Summary of Case Text", lines=6)
            recommendations_output = gr.HTML(label="Recommended Papers")
    csv_output = gr.File(label="📥 Download Recommendations CSV")
    download_topics_button = gr.Button("📘 Download Past Topics")
    topics_csv_output = gr.File(label="📥 Topics CSV")

    # Connect button
    submit_btn.click(
            fn=run_agent_ui,
            inputs=[user_input, domain, subdomain],
            outputs=[summary_output, recommendations_output, csv_output]  # 3 outputs
    )
    download_topics_button.click(
      fn=lambda: mcp_server.export_topics_csv(),
      outputs=topics_csv_output
    )




# Launch
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20e9b8ef3a781c0caf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
